In [ ]:
import tensorflow as tf
import keras
import xgboost as xgb
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm
import joblib
import os
import matplotlib.pyplot as plt 
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score

In [ ]:
print("TensorFlow:", tf.__version__)
print("Keras:", keras.__version__)

In [ ]:
class CFG:
    verbose = 1  # Verbosity
    seed = 42  # Random seed
    preset = "efficientnetv2_b2_imagenet"  # Name of pretrained classifier
    image_size = [224, 224]  # Input image size
    epochs = 12 # Training epochs
    batch_size = 96  # Batch size
    lr_mode = "step" # LR scheduler mode from one of "cos", "step", "exp"
    drop_remainder = True  # Drop incomplete batches
    num_classes = 6 # Number of classes in the dataset
    num_folds = 5 # Number of folds to split the dataset
    fold = 0 # Which fold to set as validation data
    class_names = ['X4_mean', 'X11_mean', 'X18_mean',
                   'X26_mean', 'X50_mean', 'X3112_mean',]
    aux_class_names = list(map(lambda x: x.replace("mean","sd"), class_names))
    num_classes = len(class_names)
    aux_num_classes = len(aux_class_names)

In [ ]:
keras.utils.set_random_seed(CFG.seed)

In [ ]:
BASE_PATH = "/blue/esi4611/share/planttraits2024"

In [ ]:
# Train + Valid
train = pd.read_csv(f'{BASE_PATH}/train.csv')
#train['image_path'] = f'{BASE_PATH}/train_images/'+train['id'].astype(str)+'.jpeg'
train.loc[:, CFG.aux_class_names] = train.loc[:, CFG.aux_class_names].fillna(-1)
display(train.head(2))

# Test
test = pd.read_csv(f'{BASE_PATH}/test.csv')
#test['image_path'] = f'{BASE_PATH}/test_images/'+test['id'].astype(str)+'.jpeg'
FEATURE_COLS = test.columns[1:-1].tolist()
display(test.head(2))

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
image_model = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')

# Define the function to create a TensorFlow dataset for images
def create_dataset(image_paths, batch_size=128):
    def process_path(file_path):
        img = tf.io.read_file(file_path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [224, 224])
        img = preprocess_input(img)
        return img
    path_ds = tf.data.Dataset.from_tensor_slices(image_paths)
    image_ds = path_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
    image_ds = image_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return image_ds

def extract_features_with_dataset(dataset, df):
    features_list = []
    for batch_imgs in dataset:
        print(".", end="")  # Print progress
        features = image_model.predict(batch_imgs, verbose=0)
        features_list.extend(features)
    features_array = np.array(features_list)
    
    # Convert the features array into a DataFrame
    features_df = pd.DataFrame(features_array)
    
    features_df.columns = [f'feature_{i}' for i in range(features_array.shape[1])]
    
    new_df = pd.concat([df.reset_index(drop=True), features_df.reset_index(drop=True)], axis=1)
    
    return new_df

In [ ]:
train_image_folder = '/blue/esi4611/share/planttraits2024/train_images'

image_paths = [os.path.join(train_image_folder, f"{img_id}.jpeg") for img_id in train['id']]

# Create the dataset
image_dataset = create_dataset(image_paths)

# Extract features and directly insert them into the DataFrame as separate columns
train = extract_features_with_dataset(image_dataset, train)

print(train.head())

In [ ]:
#do cross-validation testing (this is relatively slow)
do_cv = True

X_full = train.drop(columns=mean_columns)
Y_full = train[mean_columns]

models = {}

for column in Y_full.columns:

    model = AdaBoostClassifier(n_estimators=n_estimators, random_state=42)
    if do_cv:
        print(f"\nDoing cross-validation scoring for {column}...")
        scores = cross_val_score(model, X_full, Y_full[column],
                                 cv=KFold(n_splits=3, shuffle=True, random_state=42),
                                 scoring='r2')        
        print(f"R^2 score for {column}: {np.mean(scores)}")
    
    #train model with all data
    print(f"Training model for {column}...")
    model.fit(X_full, Y_full[column])
    models[column] = model

In [ ]:
# Test
test_paths = test_df.image_path.values
test_features = scaler.transform(test_df[FEATURE_COLS].values) 
test_ds = build_dataset(test_paths, test_features, batch_size=CFG.batch_size,
                         repeat=False, shuffle=False, augment=False, cache=False)

In [ ]:
pred_df = test_df[["id"]].copy()
target_cols = [x.replace("_mean","") for x in CFG.class_names]
pred_df[target_cols] = preds.tolist()

sub_df = pd.read_csv(f'{BASE_PATH}/sample_submission.csv')
sub_df = sub_df[["id"]].copy()
sub_df = sub_df.merge(pred_df, on="id", how="left")
sub_df.to_csv("submission.csv", index=False)
sub_df.head()